## Pyspark pour la manipulation et l'analyse de gros données



In [1]:
!pip install pyspark -q
!pip install findspark -q

In [2]:
# Creation d'une session Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final Project").getOrCreate()

In [3]:
import os

# Chemin absolu du dossier courant
chemin_courant = os.getcwd()
print("Chemin du dossier courant :", chemin_courant)

Chemin du dossier courant : /content


#### Telechargement des données

In [ ]:
# Importer depuis le local
# from google.colab import files
# uploaded = files.upload()

In [3]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2025-05-05 11:12:48--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.140.205, 52.206.68.26
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [              <=>   ]   1.83G  1.09MB/s    in 23m 51s 

2025-05-05 11:36:40 (1.31 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1962797304]



In [4]:
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv

#### Ingestion des données

In [5]:
# Lecture du fichier
df = spark.read.csv(
    "reported-crimes.csv",
    header = True
)
df.show(5)

+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13311263|   JG503434|07/29/2022 03:39:...|     023XX S TROY ST|1582|OFFENSE INVOLVING...|   CHILD PORNOGRAPHY|           RE

In [ ]:
df.count()

8305598

In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Type des variables
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



#### Manipulation des données

In [ ]:
from pyspark.sql.functions import to_timestamp, col
df = df.withColumn(
    'Date',
    to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a')
)
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
df.show(5)

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13311263|   JG503434|2022-07-29 03:39:00|     023XX S TROY ST|1582|OFFENSE INVOLVING...|   CHILD PORNOGRAPHY|           RESIDE

In [ ]:
df.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [ ]:
df.select(col('Description')).show(5)

+--------------------+
|         Description|
+--------------------+
|   CHILD PORNOGRAPHY|
|MANUFACTURE / DEL...|
|      NON-AGGRAVATED|
|          TO VEHICLE|
|           OVER $500|
+--------------------+
only showing top 5 rows



In [ ]:
# Possibilité de selectionner plus d'une colonne
df.select('Date','Description','Arrest').show(5)


+-------------------+--------------------+------+
|               Date|         Description|Arrest|
+-------------------+--------------------+------+
|2022-07-29 03:39:00|   CHILD PORNOGRAPHY|  true|
|2023-01-03 16:44:00|MANUFACTURE / DEL...|  true|
|2017-08-26 10:00:00|      NON-AGGRAVATED| false|
|2023-09-06 17:00:00|          TO VEHICLE| false|
|2023-09-06 11:00:00|           OVER $500| false|
+-------------------+--------------------+------+
only showing top 5 rows



In [ ]:
# On va creer une nouvelle colonne dans notre DataFrame
from pyspark.sql.functions import lit

In [ ]:
df.withColumn('One', lit(1)).show(10)

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|One|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|13311263|   JG503434|2022-07-29 03:39:00|     023XX S TROY ST|1582|OFFENSE INVOLVING...|   CHILD PORNOGRAPHY|     

In [ ]:
# Crimes signalés le 25-12-2023
one_day = df.filter(col('Date') == lit('2023-12-25'))
one_day

DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

In [ ]:
one_day.count()

25

In [ ]:
one_day.show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13318475|   JG553993|2023-12-25 00:00:00| 078XX S LANGLEY AVE|0810|               THEFT|           OVER $500|              STR

In [ ]:
# Pour verifier s'il y'a des duplications
one_day.distinct().count()

25

In [ ]:
df.distinct().count()

8305598

In [ ]:
# On va compter Primart Type suivant ses modalités
df.groupBy('Primary Type').count().show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|OFFENSE INVOLVING...|  59583|
|CRIMINAL SEXUAL A...|  10554|
|            STALKING|   5865|
|PUBLIC PEACE VIOL...|  54232|
|           OBSCENITY|    925|
|               ARSON|  14235|
|   DOMESTIC VIOLENCE|      1|
|            GAMBLING|  14653|
|   CRIMINAL TRESPASS| 224121|
|             ASSAULT| 553315|
|LIQUOR LAW VIOLATION|  15289|
| MOTOR VEHICLE THEFT| 422248|
|               THEFT|1760904|
|             BATTERY|1513221|
|             ROBBERY| 311585|
|            HOMICIDE|  13741|
|           RITUALISM|     24|
|    PUBLIC INDECENCY|    214|
| CRIM SEXUAL ASSAULT|  27330|
|   HUMAN TRAFFICKING|    133|
+--------------------+-------+
only showing top 20 rows



In [ ]:
df.select("Arrest").distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



In [ ]:
df.filter(col("Arrest") == "true").count() / df.select("Arrest").count()

0.25367950627998126

25,36% des crimes signalés ont conduit à des arrestations

In [ ]:
# Les 3 principaux lieux pour les crimes signalés
n_location = df.groupBy("Location Description").count()
n_location.show()

+--------------------+------+
|Location Description| count|
+--------------------+------+
|   RAILROAD PROPERTY|    16|
|SCHOOL - PRIVATE ...|  1403|
|AIRPORT TERMINAL ...|  2633|
|VEHICLE - COMMERCIAL|  1189|
|EXPRESSWAY EMBANK...|     1|
|RESIDENCE - YARD ...| 14329|
|POLICE FACILITY/V...| 18560|
|               MOTEL|     7|
|CHA PARKING LOT /...|  2201|
|            SIDEWALK|755787|
|AIRPORT TERMINAL ...|   146|
|PUBLIC GRAMMAR SC...|     2|
|OTHER RAILROAD PR...|   676|
|CTA GARAGE / OTHE...| 10276|
|            CAR WASH|  3572|
|   TRUCKING TERMINAL|     1|
|    AIRPORT/AIRCRAFT| 16296|
|            HOSPITAL|    18|
|    FEDERAL BUILDING|  1055|
|MEDICAL/DENTAL OF...|  7429|
+--------------------+------+
only showing top 20 rows



In [ ]:
# On va ordonner par ordre decroissant
n_location.orderBy('count', ascending=False).show()

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|2169606|
|           RESIDENCE|1369179|
|           APARTMENT| 978842|
|            SIDEWALK| 755787|
|               OTHER| 269961|
|PARKING LOT/GARAG...| 202939|
|               ALLEY| 184804|
|  SMALL RETAIL STORE| 165152|
|SCHOOL, PUBLIC, B...| 146370|
|          RESTAURANT| 139080|
|    RESIDENCE-GARAGE| 135308|
|VEHICLE NON-COMME...| 131532|
|RESIDENCE PORCH/H...| 124170|
|    DEPARTMENT STORE| 109026|
|  GROCERY FOOD STORE| 103881|
|         GAS STATION|  92644|
|RESIDENTIAL YARD ...|  75140|
|COMMERCIAL / BUSI...|  67981|
|       PARK PROPERTY|  62405|
|CHA PARKING LOT/G...|  56099|
+--------------------+-------+
only showing top 20 rows



In [7]:
# Activité non criminelle la moins signalée
df.groupBy('Primary Type').count().orderBy('count', ascending=True).show()

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|   DOMESTIC VIOLENCE|    1|
|NON-CRIMINAL (SUB...|    9|
|           RITUALISM|   24|
|      NON - CRIMINAL|   38|
|   HUMAN TRAFFICKING|  133|
|OTHER NARCOTIC VI...|  157|
|        NON-CRIMINAL|  189|
|    PUBLIC INDECENCY|  214|
|           OBSCENITY|  926|
|CONCEALED CARRY L...| 1529|
|        INTIMIDATION| 5033|
|            STALKING| 5865|
|          KIDNAPPING| 7454|
|CRIMINAL SEXUAL A...|10566|
|            HOMICIDE|13742|
|               ARSON|14239|
|            GAMBLING|14654|
|LIQUOR LAW VIOLATION|15292|
|INTERFERENCE WITH...|19783|
| CRIM SEXUAL ASSAULT|27326|
+--------------------+-----+
only showing top 20 rows



In [14]:
# Les 30 premieres observations
df.select('Primary Type').show(30, truncate=30)

+--------------------------+
|              Primary Type|
+--------------------------+
|OFFENSE INVOLVING CHILDREN|
|                 NARCOTICS|
|       CRIM SEXUAL ASSAULT|
|           CRIMINAL DAMAGE|
|                     THEFT|
|                  BURGLARY|
|               SEX OFFENSE|
|                   ROBBERY|
|       MOTOR VEHICLE THEFT|
|                  BURGLARY|
|                   BATTERY|
|                   BATTERY|
|                  HOMICIDE|
|                  HOMICIDE|
|                   ROBBERY|
|                   ROBBERY|
|                   ROBBERY|
|                  BURGLARY|
|                   ROBBERY|
|   CRIMINAL SEXUAL ASSAULT|
|OFFENSE INVOLVING CHILDREN|
|OFFENSE INVOLVING CHILDREN|
|OFFENSE INVOLVING CHILDREN|
|             OTHER OFFENSE|
|OFFENSE INVOLVING CHILDREN|
|               SEX OFFENSE|
|                   ROBBERY|
|               SEX OFFENSE|
|   CRIMINAL SEXUAL ASSAULT|
|         WEAPONS VIOLATION|
+--------------------------+
only showing t

In [7]:
n_crime_type = df.select('Primary Type').distinct().count()
n_crime_type

36

In [9]:
# Afficher toutes les types de Crime
df.select('Primary Type').distinct().show(n_crime_type, truncate=False)

+---------------------------------+
|Primary Type                     |
+---------------------------------+
|OFFENSE INVOLVING CHILDREN       |
|CRIMINAL SEXUAL ASSAULT          |
|STALKING                         |
|PUBLIC PEACE VIOLATION           |
|OBSCENITY                        |
|ARSON                            |
|DOMESTIC VIOLENCE                |
|GAMBLING                         |
|CRIMINAL TRESPASS                |
|ASSAULT                          |
|LIQUOR LAW VIOLATION             |
|MOTOR VEHICLE THEFT              |
|THEFT                            |
|BATTERY                          |
|ROBBERY                          |
|HOMICIDE                         |
|RITUALISM                        |
|PUBLIC INDECENCY                 |
|CRIM SEXUAL ASSAULT              |
|HUMAN TRAFFICKING                |
|INTIMIDATION                     |
|PROSTITUTION                     |
|DECEPTIVE PRACTICE               |
|CONCEALED CARRY LICENSE VIOLATION|
|SEX OFFENSE                

In [15]:
from pyspark.sql.functions import col
# On va creer une DataFrame des activités non criminelles
nc = df.filter(
    (col('Primary Type') == 'NON - CRIMINAL') | (col('Primary Type') == 'NON-CRIMINAL') | (col('Primary Type') == 'NON-CRIMINAL (SUBJECT SPECIFIED)')
)
nc.show()

+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13710892|   JJ103416|01/04/2025 02:35:...| 011XX W SHERIDAN RD|1481|        NON-CRIMINAL|CONCEALED CARRY L...|             

In [16]:
nc.count()

236

In [17]:
# Activité non criminelle la plus signalée
nc.groupBy('Description').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|         Description|count|
+--------------------+-----+
|       LOST PASSPORT|  111|
|   FOID - REVOCATION|   76|
|CONCEALED CARRY L...|   25|
|NOTIFICATION OF C...|    9|
|NOTIFICATION OF S...|    8|
|      FOUND PASSPORT|    4|
|GUN OFFENDER NOTI...|    3|
+--------------------+-----+



l'activité non criminelle la plus signalée reste Lost passport


In [19]:
# Pour connaitre d'autres fonctions de ce module
from pyspark.sql import functions
dir(functions)

['Any',
 'ArrayType',
 'Callable',
 'Column',
 'DataFrame',
 'DataType',
 'Dict',
 'Iterable',
 'JVMView',
 'List',
 'Optional',
 'PandasUDFType',
 'PySparkTypeError',
 'PySparkValueError',
 'SparkContext',
 'StringType',
 'StructType',
 'TYPE_CHECKING',
 'Tuple',
 'Type',
 'Union',
 'UserDefinedFunction',
 'UserDefinedTableFunction',
 'ValuesView',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_create_column_from_literal',
 '_create_lambda',
 '_create_py_udf',
 '_create_py_udtf',
 '_from_numpy_type',
 '_get_jvm_function',
 '_get_lambda_parameters',
 '_invoke_binary_math_function',
 '_invoke_function',
 '_invoke_function_over_columns',
 '_invoke_function_over_seq_of_columns',
 '_invoke_higher_order_function',
 '_options_to_str',
 '_test',
 '_to_java_column',
 '_to_seq',
 '_unresolved_named_lambda_variable',
 'abs',
 'acos',
 'acosh',
 'add_months',
 'aes_decrypt',
 'aes_encrypt',
 'aggregate',
 'any_value',
 'approxC

In [20]:
len(dir(functions))

484